# Geometry Fixer for Vector Files

## Description
This Python script reads a vector file (e.g., Shapefile, GeoJSON), fixes invalid geometries, and saves the corrected geometries to a new output file. It uses the `geopandas` and `shapely` libraries to handle geospatial data and repair geometry errors effectively.

The script is designed to address common geometry issues such as:
- Self-intersecting polygons
- Invalid topology
- Other minor geometry inconsistencies

## Requirements
- Python 3.x
- `geopandas` and `shapely` libraries

## Author
- Rubén Crespo Ceballos

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiPolygon
import os

In [ ]:
"""Input variables"""
input_path = r""
# In this case we have the output on the same folder.

In [ ]:
def get_vector_file_list(path):
    """
    Get a list of the vector files inside the folder
    Parameters:
    - path (str): path of the folder with the resources.

    Returns:
    - File_list (list). list of the resources.
    """
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        # "anat" is just to get here necessary ones
        if file.endswith(".shp"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def fix_geometries(input_file, output_file):
    """
    Reads a vector file, fixes invalid geometries, and saves the corrected file.

    Parameters:
        input_file (str): The path to the input vector file (e.g., Shapefile or GeoJSON).
        output_file (str): The path to save the output file with fixed geometries.

    Returns:
        None: The function saves the fixed file to the specified output path.
    """
    # Read the vector file into a GeoDataFrame
    gdf = gpd.read_file(input_file)
    
    # Function to fix individual geometries
    def fix_geometry(geom):
        """
        Fixes an individual geometry if it is invalid.

        Parameters:
            geom (shapely.geometry.base.BaseGeometry): A geometry object to be fixed.

        Returns:
            shapely.geometry.base.BaseGeometry: The fixed geometry object.
        """
        if geom.is_valid:
            return geom
        else:
            return geom.buffer(0)  # Buffer(0) often fixes geometry errors
    
    # Fix all geometries in the GeoDataFrame
    gdf['geometry'] = gdf['geometry'].apply(fix_geometry)
    
    # Handle MultiPolygons, if needed
    gdf['geometry'] = gdf['geometry'].apply(
        lambda geom: MultiPolygon([geom]) if geom.geom_type == 'Polygon' else geom
    )
    
    # Save the corrected GeoDataFrame to a new file
    gdf.to_file(output_file)
    print(f"Fixed geometries saved to {output_file}")


In [ ]:
vector_files_list = get_vector_file_list(input_path)
for file in vector_files_list:
    output_file = file.replace(".tif", "_fixed_geometries.tif")
    fix_geometries(file, output_file)